In [ ]:
# Mnist라는 숫자를 그림으로 표현한 것을 학습시킬 것

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
z_dim=128

In [ ]:
# autoencoder model 정의
class Autoencoder(nn.Module):
  def __init__(self, z_dim=128):
    super(Autoencoder, self).__init__()
    self.encoder = torch.nn.Sequential(
        torch.nn.Linear(28*28, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, z_dim),
    )
    self.decoder = torch.nn.Sequential(
        torch.nn.Linear(z_dim, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 28*28),
        nn.Sigmoid(),
    )

  def forward(self, x):
    x = x.view(x.size(0), -1)
    x = self.encoder(x)
    x = self.decoder(x)
    x = x.view(x.size(0), 28, 28)
    return x


In [ ]:
# MNIST dataset 불러오기! > 아래의 내용은 기본적으로 tensor로 변환만 하고 있음 > MNIST는 이미 데이터 기본 전처리가 되어있기 때문
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
train_dataset = MNIST(route='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(route='./data', train=False, download=True, transform=transform)

In [ ]:
# 각 클래스별로 저장된 파일을 담을 dict 만들기
samples = {}
for i in range(10):
  samples[i] = None

cpt = 0

# 훈련용 데이터에 들어있는 정보를 클래스별로 저장해두기
for data, target in train_dataset:
  if samples[target] is None:
    samples[target] = data
    cpt += 1
    if cpt == 10:
      break

# sample 그리기
fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for i in range(10):
  axes[i].imshow(samples[i][0], cmap='gray')
  axes[i].set_title(f'Class {i}')
  axes[i].axis('off')

plt.show()

In [ ]:
# DataLoader 만들기
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
#
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [ ]:
from torch import optim
#AutoEncoder model을 이용하여 모델 객체 만들고
# 특정 device로 불러오기 - cpu, gpu 중에 선택
model = Autoencoder().to(device)

# optimizer 객체 만들기
# 일반적으로 Adam을 많이 사용함..
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# loss 정의
criterion = nn.MSELoss()

In [ ]:
# Tensor의 모습 확인할 것 - 이것은 코드를 작성하는 과정에서만 잠시 임의로 쓰이는 부분
# I = torch.rand((1,1,28,28)).cuda()
# with torch.no_grad():
#  print(model(I).shape)

In [ ]:
# 훈련
num_epochs = 100
train_losses = []
for epoch in range[num_epochs]:
  running_loss = 0.0
  for images, _ in train_loader:
    optimizer.zero_grad()
    outputs = model(images.cuda())
    loss = nn.functional.binary_cross_entropy(outputs.reshape(-1, 1), images.cuda().reshape(-1, 1), reductions='sum')
    loss.backward()
    optimizer.step()
    running_loss += loss.item() * images.size(0)
  train_loss = running_loss / len(train_loader.dataset)
  train_losses.append(train_loss)
  if epoch % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')


In [ ]:
# random images를 선택하고 그 테스트 이미지들을 재구성하기
num_images = 5

selected_indices = torch.randint(len(test_dataset), size=(num_images,))
reconstructed_images = []
original_images = []
for idx in selected_indices:
  image, _ = test_dataset[idx]
  original_images.append(image[0])
  with torch.no_grad():
    reconstructed_image = model(image.cuda().unsqueeze(0))
    reconstructed_images.append(reconstructed_image.squeeze().detach().cpu().numpy().reshape(28,28))

# 원본 그림, 복원된 그림 확인하기
fig, axes = plt.subplots(num_images, 2, figsize=(8, 2+num_images))
for i in range(num_images):
  axes[i, 0].imshow(original_images[i], cmap='gray')
  axes[i, 0].set_title('Original Image')
  axes[i, 0].axis('off')

  axes[i, 1].imshow(reconstructed_images[i], cmap='gray')
  axes[i, 1].set_title('Reconstructed Image')
  axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot of latent space with respective class colors
latent_points = []
labels = []
with torch.no_grad():
  for images, targets in test_loader:
    latent = model.encoder(images.cuda().view(images.size(0), -1))
    latent_points.extend(latent.detach().cpu().numpy())
    labels.extend(targets.numpy())


In [ ]:
latent_points = np.array(latent_points)
labels = np.array(labels)

In [1]:
from sklearn.manifold import TSME
# t-SNE을 적용하여 latent 표현을 2차원으로 차원을 줄이기
tsne = TSNE(n_components=2, random_state=0)
latent_2d = tsne.fit_transform(latent_points)

# 2차원으로 표현된 latent space 확인하기
plt.figure(figsize=(10, 8))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=test_dataset.targets, cmap='tab10', alpha=0.5)
plt.colorbar(label('Digit class'))
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE Visualization of latent space')
plt.show()

ImportError: cannot import name 'TSME' from 'sklearn.manifold' (/usr/local/lib/python3.10/dist-packages/sklearn/manifold/__init__.py)

In [ ]:
# 2군데의 랜덤한 latent point를 골라보기
selected_indices = np.random.choice(len(latent_points), 2, replace=false)
selected_latent = latent_points[selected_indices]
selected_latent_2d = latent_2d[selected_indices]
selected_labels = lables[selected_indices]

In [ ]:
# latent space에서 class color를 입혀 어느 클래스고 어느 latent 공간에 있는지 확인해보기
plt.scatter(latent_2d[:, 0], latent_2d[:, 1], label='Selected Latent Points')
plt.scatter(selected_latent_2d[:, 0], selected_latent_2d[:, 1], label='Selected Latent Points')
plt.xlabel('Latent Dim 1')
plt.ylabel('Latent Dim 2')
plt.title('Latent Space with Selected Latent Points')
plt.legend()
plt.show()

In [ ]:
latent1, latent2 = torch.from_numpy(selected_latent)

interpolation_points = torch.zeros((10, z_dim))
for i in range(10):
  interpolation_points[i] = latent1 + (latent2 - latent1) * 1 / 9


In [ ]:
# 선택된 곳의 latent point를 decode해보기
with torch.no_grad():
  decoded_images = model.decoder(interpolation_points.cuda()).detach().cpu().numpy().reshape(-1, 28, 28)

In [ ]:
# decoded images
fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for i in range(10):
  axes[i].imshow(decoded_images[i], cmap='gray')
  axes[i].axis('off')

plt.show()